# Preprocessing and Manipulation of Data Sets
This note contains steps for data preprocessing and manipulation. It takes raw data and raw stimuli table as input (csv starts with number 00), and output a final data set after merging all other data sets in csv format (starts with number 01).

Import relevant packages.

In [145]:

import pandas as pd
import numpy as np
from pyro import distributions as dist

## Read data sets

In [146]:
# Read dataset
df_slider = pd.read_csv('00-slider-data-raw.csv')
df_slider_for_exclusion = pd.read_csv('00-slider-data-raw.csv')
df_slider_subj_info = pd.read_csv('00-slider-subj-info.csv')

df_stimuli = pd.read_csv('00-stimuli-table-raw.csv') # before: df_stimuli
df_production = pd.read_csv('00-production-data-raw.csv',encoding='latin-1') # before: df_production

In [147]:
# Merge dataframes
df_slider = pd.merge(df_slider, df_stimuli, on=['list', 'conditions','item'], how='left')
df_production = pd.merge(df_production, df_stimuli, on=['list', 'conditions','item'], how='left')

## Preprocessing of Slider data 

Inspection of NA values.

In [148]:
# Mark slider value of -1 as NaN
df_slider['slider_value'] = df_slider['slider_value'].replace("-1", np.nan)

# show proportion of NaN values
print(f"Proportion of NaN values of slider value: {df_slider['slider_value'].isna().sum()/len(df_slider)}")

Proportion of NaN values of slider value: 0.08503703703703704


In [149]:
# How many missing values per column?
print(f"Number of missing values per column: {df_slider.isnull().sum()}")

# Show rows with missing values with colname "list"
df_slider[df_slider['list'].isnull()]


Number of missing values per column: id                    0
item                  0
conditions            0
read_time             0
image_error           0
slider_value       2296
leftright_trial       0
list                 89
trials               89
D                  9060
C                  9057
F                  9061
size_A               89
size_B               89
size_C               89
size_D               89
size_E               89
size_F               89
color_A              89
color_B              89
color_C              89
color_D              89
color_E              89
color_F              89
form_A               89
form_B               89
form_C               89
form_D               89
form_E               89
form_F               89
noun                 89
sharpness            89
dtype: int64


,id,item,conditions,read_time,image_error,slider_value,leftright_trial,list,trials,D,...,color_E,color_F,form_A,form_B,form_C,form_D,form_E,form_F,noun,sharpness
1547,6241ee95e86a3,9,brcf,[6492],False,1right,5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1569,6241ee95e86a3,11,ercf,[4226],False,1left,5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1573,6241ee95e86a3,34,fzrdc,[4390],False,1left,5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1911,6241f3f13c93c,35,fbrcf,[4973],False,1right,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2647,6241fe6a604b4,31,fzrdc,[3582],False,1right,6,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24237,626167b76e314,30,fzrdf,[6333],False,1right,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24238,626167b76e314,36,fbrdc,[2977],False,1right,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24239,626167b76e314,30,fzrcf,[4059],False,1left,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24419,626168378ca1c,35,fzrdf,[3966],False,1right,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The introduced NA values are due to missing list variable in the data set. After merging, it also introduced NA to other columns. The number of missing list is 89. This matched the number of NA values in other columns. Therefore, we should drop NA values depend on the variable list. NAs in columns D, C and F are due to experiment design.

In [150]:
# Drop rows with missing values and colname "list"
df_slider = df_slider.dropna(subset=['list'])
df_slider = df_slider.dropna(subset=['slider_value'])

# Check again how many missing values per column
print(f"Number of missing values per column: {df_slider.isnull().sum()}")

Number of missing values per column: id                    0
item                  0
conditions            0
read_time             0
image_error           0
slider_value          0
leftright_trial       0
list                  0
trials                0
D                  8245
C                  8245
F                  8125
size_A                0
size_B                0
size_C                0
size_D                0
size_E                0
size_F                0
color_A               0
color_B               0
color_C               0
color_D               0
color_E               0
color_F               0
form_A                0
form_B                0
form_C                0
form_D                0
form_E                0
form_F                0
noun                  0
sharpness             0
dtype: int64


Encode slider values depend on the variable leftright_trial. This variable is a random variable of two factor levels: 1left and 1right. The slider value is reversed in condition 1left than in condition 1right. We take 1right as default, as this can map higher preference to higher slider value.

In [151]:
# Convert slider value into numeric
df_slider['slider_value'] = df_slider['slider_value'].astype('float32')

# Create a new column called 'prefer_first_1st' based on the 'leftright_trial' and 'slider_value' columns
df_slider['prefer_first_1st'] = df_slider.apply(lambda x: 100 - x['slider_value'] if x['leftright_trial'] == '1left' else x['slider_value'], axis=1)

# Get an overview of slider value
df_slider['prefer_first_1st'].describe()  

count    24615.000000
mean        61.763721
std         42.717204
min          0.000000
25%          7.000000
50%         87.000000
75%        100.000000
max        100.000000
Name: prefer_first_1st, dtype: float64

Drop unnecessary columns.

In [152]:
df_slider = df_slider.drop(columns=['slider_value', 'leftright_trial', 'D', 'C', 'F', 'read_time', 'image_error', 'noun'])

Exclude PPs. We have five criteria for exclusion.

- Exclude pps with more than than 3 missing list
- Exclude pps with more than 2 SE away from mean experiment duration 
- Exclude pps with more than 3 false responses to control items (item 34-38, in condition ferdc, fercf, ferdf, fzrcf, fzrdc, fzrdf, item 38 in condition fzrcf)
- Exclude more than 3 none fit in experimental items

In [153]:
# Criteria 1: Exclude pps with more than than 3 missing list
filtered_data = df_slider_for_exclusion[df_slider_for_exclusion['list'].isna()]
cross_tab = pd.crosstab(index=filtered_data['id'], columns='count')
result = cross_tab[cross_tab['count'] > 3]
exclude_1 = result.index.tolist()
print(f"Number of participants to exclude based on criteria 1: {len(exclude_1)}")

# Criteria 2: Exclude pps with more than two SE away from mean experiment duration 
sd = df_slider_subj_info['time_in_minutes'].std()
mean_time = df_slider_subj_info['time_in_minutes'].mean()
x = mean_time + 2 * sd
y = mean_time - 2 * sd
filtered_subj_info = df_slider_subj_info[(df_slider_subj_info['time_in_minutes'] < y) | (df_slider_subj_info['time_in_minutes'] > x)]
exclude_2 = filtered_subj_info.id.tolist()
print(f"Number of participants to exclude based on criteria 2: {len(exclude_2)}")

# Criteria 3: Exclude pps with more than 3 false responses to control items
df_slider_for_exclusion['none_fits'] = df_slider_for_exclusion.apply(lambda row: 1 if row['slider_value'] == -1 or pd.isna(row['list']) else 0, axis=1)

filtered_data_1 = df_slider_for_exclusion[
    (df_slider_for_exclusion['item'].isin([34, 35, 36, 37, 38])) &
    (df_slider_for_exclusion['conditions'].isin(["ferdc", "fercf", "ferdf", "fzrdc", "fzrdf"])) &
    (df_slider_for_exclusion['none_fits'] == 1)
]

cross_tab_criteria_3_1 = pd.crosstab(index=filtered_data_1['id'], columns='count')
exclude_criteria_3_1 = cross_tab_criteria_3_1[cross_tab_criteria_3_1['count'] > 3].index.tolist()

filtered_data_2 = df_slider_for_exclusion[
    (df_slider_for_exclusion['item'].isin([38])) &
    (df_slider_for_exclusion['conditions'].isin(["fzrcf"])) &
    (df_slider_for_exclusion['none_fits'] == 0)
]
cross_tab_criteria_3_2 = pd.crosstab(index=filtered_data_2['id'], columns='count')
exclude_criteria_3_2 = cross_tab_criteria_3_2[cross_tab_criteria_3_2['count'] > 3].index.tolist()
print(f"Number of participants to exclude based on criteria 3: {len(exclude_criteria_3_1 + exclude_criteria_3_2)}")

# Criteria 4: Exclude more than 3 none fit in experimental items
experiment_items = df_slider_for_exclusion[df_slider_for_exclusion['conditions'].str[0:1] != 'f']
filtered_data_criteria_4 = experiment_items[
    (df_slider_for_exclusion['none_fits'] == 1)
]
cross_tab_criteria_4 = pd.crosstab(index=filtered_data_criteria_4['id'], columns='count')

exclude_4 = cross_tab_criteria_4[cross_tab_criteria_4['count'] > 3].index.tolist()
print(f"Number of participants to exclude based on criteria 4: {len(exclude_4)}")

# Combine all exclusion criteria and remove duplicates
exclude_all= list(set(exclude_1 + exclude_2 + exclude_criteria_3_1 + exclude_criteria_3_2 + exclude_4))

# Print how many participants to exclude
print(f"Number of participants to exclude: {len(exclude_all)}")


# Print how many participants are in the dataset before exclusion
print(f"Number of participants in the dataset: {len(df_slider_for_exclusion.id.unique())}")
# Exclude pps
df_slider = df_slider[~df_slider['id'].isin(exclude_all)]
# Print how many participants are in the dataset after exclusion
print(f"Number of participants in the dataset: {len(df_slider.id.unique())}")

Number of participants to exclude based on criteria 1: 8
Number of participants to exclude based on criteria 2: 8
Number of participants to exclude based on criteria 3: 2
Number of participants to exclude based on criteria 4: 6
Number of participants to exclude: 16
Number of participants in the dataset: 150
Number of participants in the dataset: 134


/var/folders/jc/6ctllcx143xbrgx01x69skg80000gn/T/ipykernel_59188/2358079012.py:40: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  filtered_data_criteria_4 = experiment_items[


Subset filler items from the dataframe and only keep experiment items.

In [154]:
# Create a new DataFrame called 'data_filler' by subsetting the rows where the first character of the 'conditions' column is 'f'
data_slider_filler = df_slider[df_slider['conditions'].str[0:1] == 'f']

# Remove the rows where the first character of the 'conditions' column is 'f' from the original DataFrame 'df_slider'
df_slider = df_slider[df_slider['conditions'].str[0:1] != 'f'].reset_index(drop=True)

Encode conditions from German to English.

In [155]:
# Create a new column called 'combination' based on the 'conditions' column
df_slider['combination'] = df_slider['conditions'].str[2:4].apply(lambda x: 'color_form' if x == 'cf' else 'dimension_color' if x == 'dc' else 'dimension_form')
df_slider['combination'] = df_slider['combination'].astype('category')

# Create a new column called 'relevant_property' based on the 'conditions' column
df_slider['relevant_property'] = df_slider['conditions'].str[0:1].apply(lambda x: 'first' if x == 'e' else 'second' if x == 'z' else 'both')
df_slider['relevant_property'] = df_slider['relevant_property'].astype('category')

# Encode the index of the 'id' column starting from 1
df_slider['id'] = pd.factorize(df_slider['id'])[0] + 1

In [156]:
# Take a final look at the DataFrame
print(df_slider.head())


   id  item conditions  list  trials  size_A  size_B  size_C  size_D  size_E  \
0   1    12       zrdf   5.0     9.0    10.0     9.0     9.0     9.0     9.0   
1   1     1       erdf   5.0     6.0    10.0     4.0     4.0     2.0     2.0   
2   1    19       zrcf   5.0     7.0     3.0    10.0     2.0    10.0     3.0   
3   1    10       zrdc   5.0     8.0     9.0     9.0    10.0     9.0     9.0   
4   1    13       erdc   5.0     5.0    10.0     5.0     3.0     1.0     1.0   

   ...    form_A    form_B    form_C    form_D    form_E    form_F sharpness  \
0  ...      star     heart     heart     heart     heart     heart   blurred   
1  ...  triangle  triangle  triangle  triangle  triangle   diamond   blurred   
2  ...   diamond  triangle  triangle  triangle  triangle  triangle   blurred   
3  ...     heart   quadrat   quadrat   diamond  triangle   quadrat   blurred   
4  ...    circle   quadrat   quadrat      star   quadrat     heart   blurred   

  prefer_first_1st      combination re

## Preprocessing of production data

Exclude pps. We have X criteria for exclusion:
- Criteria 1: Exclude pps with more than three s.e. away from mean experiment time
- Criteria 2: Exclude pps with more than 80% identical responses (Number of total trials: 135, 80% of these are 108)
- Criteria 3: Exclude pps with more than 10% NA responses

In [ ]:
df_production_subj_info = pd.read_csv('00-production-subj-info.csv')
df_production_for_exclusion = df_production.copy()

# Criteria 1: Exclude pps with more than two SE away from mean experiment duration 
sd = df_production_subj_info['time_in_minutes'].std()
mean_time = df_production_subj_info['time_in_minutes'].mean()
x = mean_time + 2 * sd
y = mean_time - 2 * sd
filtered_subj_info = df_production_subj_info[(df_production_subj_info['time_in_minutes'] < y) | (df_production_subj_info['time_in_minutes'] > x)]
exclude_1 = filtered_subj_info.id.tolist()
print(f"Number of participants to exclude due to criteria 1: {len(exclude_1)}")

# Criteria 2: Exclude pps with more than 60% identical responses
# Show how many trials each participant has
cross_tab_trials = pd.crosstab(index = df_production['id'], columns='count')
trials_per_pp = cross_tab_trials['count'].unique()[0]
cutoff = 0.8 * trials_per_pp
cross_tab_annotation = pd.crosstab(index = [df_production['id'], df_production['annotation']], columns='count').reset_index()
filtered_data_1 = cross_tab_annotation[cross_tab_annotation['count'] > cutoff]
exclude_2 = filtered_data_1['id'].unique().tolist()
print(f"Number of participants to exclude due to criteria 2: {len(exclude_2)}")

# Criteria 3: Exclude pps with more than 10% NA responses
data_na = df_production[df_production['annotation'].isna()]
id_counts_na = data_na['id'].value_counts()
cutoff = 0.1 * trials_per_pp
exclude_3 = id_counts_na[id_counts_na > cutoff].index.tolist()
print(f"Number of participants to exclude due to criteria 3: {len(exclude_3)}")

# Combine all exclusion criteria and remove duplicates
exclude_all = list(set(exclude_1 + exclude_2 + exclude_3))

# Print how many participants to exclude
print(f"Number of participants to exclude: {len(exclude_all)}")

# Print how many participants are in the dataset before exclusion
print(f"Number of participants in the dataset: {len(df_production_for_exclusion.id.unique())}")
# Exclude pps
df_production = df_production[~df_production['id'].isin(exclude_all)]
# Print how many participants are in the dataset after exclusion
print(f"Number of participants in the dataset: {len(df_production.id.unique())}")


Number of participants to exclude due to criteria 1: 8
Number of participants to exclude due to criteria 2: 25
Number of participants to exclude due to criteria 3: 7
Number of participants to exclude: 36
Number of participants in the dataset: 149
Number of participants in the dataset: 113


Remove NAs.

In [158]:
# Show percentage of missing values in the 'annotation' column
print(f"Percentage of missing values in the 'annotation' column: {df_production['annotation'].isna().sum()/len(df_production)}")

# Remove NAs from the 'annotation' column
df_production = df_production.dropna(subset=['annotation'])


Percentage of missing values in the 'annotation' column: 0.026377597109304426


In [18]:
# Encode the index of the 'id' column starting from 1
df_production['id'] = pd.factorize(df_production['id'])[0] + 1

Remove filler items. Encode conditions. And encode ids.

In [138]:
# Encode the index of the 'id' column starting from 1
df_production['id'] = pd.factorize(df_production['id'])[0] + 1

# Remove filler items from the data set
df_production = df_production[df_production['conditions'].str[0:1] != 'f'].reset_index(drop=True)

# Create a new column called 'combination' based on the 'conditions' column
df_production['combination'] = df_production['conditions'].str[2:4].apply(lambda x: 'color_form' if x == 'cf' else 'dimension_color' if x == 'dc' else 'dimension_form')
df_production['combination'] = df_production['combination'].astype('category')

# Create a new column called 'relevant_property' based on the 'conditions' column
df_production['relevant_property'] = df_production['conditions'].str[0:1].apply(lambda x: 'first' if x == 'e' else 'second' if x == 'z' else 'both')
df_production['relevant_property'] = df_production['relevant_property'].astype('category')

# Drop unnecessary columns
df_production = df_production.drop(columns=['response', 'D', 'C', 'F', 'read_time', 'image_error', 'noun'])


In [139]:
# Take a final look at the DataFrame
print(df_production.head())

   id  item conditions  list annotation  trials  size_A  size_B  size_C  \
0   1     5       ercf     1          C       4       2       2       9   
1   1    15       zrdc     1          C       8       9      10      10   
2   1    18       zrdc     1          C       8       9      10       9   
3   1     1       brdc     1        DCF       2      10      10       3   
4   1    10       brcf     1         CF       1       2      10      10   

   size_D  ...  color_F    form_A   form_B    form_C    form_D   form_E  \
0       9  ...    black   quadrat  quadrat   quadrat   quadrat  quadrat   
1      10  ...    green     heart    heart   quadrat   quadrat  quadrat   
2      10  ...    brown   quadrat    heart  triangle  triangle   circle   
3       2  ...   orange  triangle  quadrat   quadrat    circle  diamond   
4       9  ...     blue  triangle  diamond  triangle  triangle  diamond   

     form_F sharpness      combination relevant_property  
0      star     sharp       color_form 

## Simplify properties of object into a binary category for modeling

In [141]:
# Re-encode the data such that there are only two categories for colors and forms while keeping the same pattern (experimental manipulation)
# ... in order for simplicity of the modeling
def encode_color(line):
    if (line.conditions == 'ercf') or (line.conditions == 'zrdc'): # color is the most discriminative feature
        line.color_A = "blue"
        line.color_B = "red"
        line.color_C = "red"
        line.color_D = "red"
        line.color_E = "red"
        line.color_F = "red"
    elif (line.conditions == 'erdc') or (line.conditions == 'zrcf'): # size is the most discriminative feature
        line.color_A = "red"
        line.color_B = "red"
        line.color_C = "red"
        line.color_D = "red"
        line.color_E = "red"
        line.color_F = "blue"
    elif (line.conditions == 'brdc'): # both color and size are discriminative features
        line.color_A = "blue"
        line.color_B = "red"
        line.color_C = "blue"
        line.color_D = "blue"
        line.color_E = "red"
        line.color_F = "red"
    elif (line.conditions == 'brcf'): # both color and form are discriminative features
        line.color_A = "blue"
        line.color_B = "blue"
        line.color_C = "red"
        line.color_D = "red"
        line.color_E = "blue"
        line.color_F = "red"
    elif (line.conditions == 'erdf') or (line.conditions == 'zrdf') or (line.conditions == 'brdf'): # color is not relevant for discrimination and therefore randomised
        line.color_A = "blue"
        line.color_B = "blue" if dist.Bernoulli(0.7).sample() == 1 else "red"
        line.color_C = "blue" if dist.Bernoulli(0.7).sample() == 1 else "red"
        line.color_D = "blue" if dist.Bernoulli(0.7).sample() == 1 else "red"
        line.color_E = "blue" if dist.Bernoulli(0.7).sample() == 1 else "red"
        line.color_F = "blue" if dist.Bernoulli(0.7).sample() == 1 else "red"
        
    return line

def encode_form(line):
    if (line.conditions == 'zrdf') or (line.conditions == 'zrcf'): # form is the most discriminative feature
        line.form_A = "circle"
        line.form_B = "square"
        line.form_C = "square"
        line.form_D = "square"
        line.form_E = "square"
        line.form_F = "square"
    elif (line.conditions == 'erdf') or (line.conditions == 'ercf'): # size or color is the most discriminative feature
        line.form_A = "circle"
        line.form_B = "circle"
        line.form_C = "circle"
        line.form_D = "circle"
        line.form_E = "circle"
        line.form_F = "square"
    elif (line.conditions == 'brdf') or (line.conditions == 'brcf'): # both size and form are discriminative features
        line.form_A = "circle"
        line.form_B = "square"
        line.form_C = "circle"
        line.form_D = "circle"
        line.form_E = "square"
        line.form_F = "square"
    elif (line.conditions == 'erdc') or (line.conditions == 'zrdc') or (line.conditions == 'brdc'): # form is not relevant for discrimination and therefore randomised
        line.form_A = "circle"
        line.form_B = "circle" if dist.Bernoulli(0.7).sample() == 1 else "square"
        line.form_C = "circle" if dist.Bernoulli(0.7).sample() == 1 else "square"
        line.form_D = "circle" if dist.Bernoulli(0.7).sample() == 1 else "square"
        line.form_E = "circle" if dist.Bernoulli(0.7).sample() == 1 else "square"
        line.form_F = "circle" if dist.Bernoulli(0.7).sample() == 1 else "square"
    return line

In [142]:
# apply the functions
df_production_encoded = df_production.apply(encode_color,axis=1)
df_production_encoded = df_production_encoded.apply(encode_form, axis=1)
df_production_encoded.head()

,id,item,conditions,list,annotation,trials,size_A,size_B,size_C,size_D,...,color_F,form_A,form_B,form_C,form_D,form_E,form_F,sharpness,combination,relevant_property
0,1,5,ercf,1,C,4,2,2,9,9,...,red,circle,circle,circle,circle,circle,square,sharp,color_form,first
1,1,15,zrdc,1,C,8,9,10,10,10,...,red,circle,square,circle,circle,circle,circle,sharp,dimension_color,second
2,1,18,zrdc,1,C,8,9,10,9,10,...,red,circle,square,square,circle,square,circle,sharp,dimension_color,second
3,1,1,brdc,1,DCF,2,10,10,3,2,...,red,circle,square,square,circle,square,circle,sharp,dimension_color,both
4,1,10,brcf,1,CF,1,2,10,10,9,...,red,circle,square,circle,circle,square,square,sharp,color_form,both


In [143]:
# also apply the functions to the slider values
df_slider_encoded = df_slider.apply(encode_color,axis=1)
df_slider_encoded = df_slider_encoded.apply(encode_form, axis=1)
df_slider_encoded.head()

,id,item,conditions,list,trials,size_A,size_B,size_C,size_D,size_E,...,form_A,form_B,form_C,form_D,form_E,form_F,sharpness,prefer_first_1st,combination,relevant_property
0,1,12,zrdf,5.0,9.0,10.0,9.0,9.0,9.0,9.0,...,circle,square,square,square,square,square,blurred,99.0,dimension_form,second
1,1,1,erdf,5.0,6.0,10.0,4.0,4.0,2.0,2.0,...,circle,circle,circle,circle,circle,square,blurred,96.0,dimension_form,first
2,1,19,zrcf,5.0,7.0,3.0,10.0,2.0,10.0,3.0,...,circle,square,square,square,square,square,blurred,98.0,color_form,second
3,1,10,zrdc,5.0,8.0,9.0,9.0,10.0,9.0,9.0,...,circle,square,circle,circle,circle,circle,blurred,97.0,dimension_color,second
4,1,13,erdc,5.0,5.0,10.0,5.0,3.0,1.0,1.0,...,circle,square,circle,square,circle,circle,blurred,98.0,dimension_color,first


In [144]:
# Remove negative values in df_slider.prefer_first_1st
# ... first show how many are there
print(df_slider_encoded[df_slider_encoded.prefer_first_1st < 0].shape)
# and proportion of the total
print(df_slider_encoded[df_slider_encoded.prefer_first_1st < 0].shape[0] / df_slider_encoded.shape[0])
# remove them
df_slider_encoded = df_slider_encoded[df_slider_encoded.prefer_first_1st >= 0].reset_index(drop=True)

(0, 27)
0.0


## Export dataframe for further analysis

In [159]:
df_slider_encoded.to_csv('01-slider-data-preprocessed.csv', index=False)
df_production_encoded.to_csv('01-production-data-preprocessed.csv', index=False)